In [1]:
import json

## Training Script what is going on NBA dataset

Output of **preprocess**  

 * number of source features- stage 1: 3.
 * number of target features- stage 1: 0.
 * number of source features- stage 2: 3.
 * number of target features- stage 2: 0.
Building `Fields` object...
Building & saving training data...
Warning. The corpus /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/rotowire/src_train.txt is larger than 10M bytes, you can set '-max_shard_size' to process it by small shards to use less memory.
('average src size', 606, 3371)
 * saving train data shard to /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.train.1.pt.
Building & saving vocabulary...
 * reloading /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.train.1.pt.
 * tgt1 vocab size: 391.
 * tgt2 vocab size: 9902.
 * src1 vocab size: 1164.
 * src1_feat_0 vocab size: 702.
 * src1_feat_1 vocab size: 39.
 * src1_feat_2 vocab size: 4.
 * src2 vocab size: 956.
 * src2_feat_0 vocab size: 547.
 * src2_feat_1 vocab size: 33.
 * src2_feat_2 vocab size: 4.
Building & saving validation data...
Warning. The corpus /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/rotowire/src_valid.txt is larger than 10M bytes, you can set '-max_shard_size' to process it by small shards to use less memory.
('average src size', 606, 722)
 * saving valid data shard to /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.valid.1.pt.

In [2]:
from __future__ import division

import argparse
import glob
import os
import sys
import random

import torch
import torch.nn as nn
from torch import cuda

import onmt
import onmt.io
import onmt.Models
import onmt.ModelConstructor
import onmt.modules
from onmt.Utils import use_gpu
import opts

In [3]:
"Command"   

"python train.py -data $BASE/preprocess/roto -save_model $BASE/gen_model/$IDENTIFIER/roto -encoder_type1 mean -decoder_type1 pointer -enc_layers1 1 -dec_layers1 1 -encoder_type2 brnn -decoder_type2 rnn -enc_layers2 2 -dec_layers2 2 -batch_size 5 -feat_merge mlp -feat_vec_size 600 -word_vec_size 600 -rnn_size 600 -seed 1234 -start_checkpoint_at 4 -epochs 25 -optim adagrad -learning_rate 0.15 -adagrad_accumulator_init 0.1 -report_every 100 -copy_attn -truncated_decoder 100 -gpuid $GPUID -attn_hidden 64 -reuse_copy_attn -start_decay_at 4 -learning_rate_decay 0.97 -valid_batch_size 5"


'python train.py -data $BASE/preprocess/roto -save_model $BASE/gen_model/$IDENTIFIER/roto -encoder_type1 mean -decoder_type1 pointer -enc_layers1 1 -dec_layers1 1 -encoder_type2 brnn -decoder_type2 rnn -enc_layers2 2 -dec_layers2 2 -batch_size 5 -feat_merge mlp -feat_vec_size 600 -word_vec_size 600 -rnn_size 600 -seed 1234 -start_checkpoint_at 4 -epochs 25 -optim adagrad -learning_rate 0.15 -adagrad_accumulator_init 0.1 -report_every 100 -copy_attn -truncated_decoder 100 -gpuid $GPUID -attn_hidden 64 -reuse_copy_attn -start_decay_at 4 -learning_rate_decay 0.97 -valid_batch_size 5'

In [4]:
BASE='/home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data'

TRAIN_DATASET='/home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.train.1.pt'

VOCAB_FILE='/home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.vocab.pt'

In [5]:
def lazily_load_dataset(corpus_type):
    """
    Dataset generator. Don't do extra stuff here, like printing,
    because they will be postponed to the first loading time.

    Args:
        corpus_type: 'train' or 'valid'
    Returns:
        A list of dataset, the dataset(s) are lazily loaded.
    """
    assert corpus_type in ["train", "valid"]

    def lazy_dataset_loader(pt_file, corpus_type):
        dataset = torch.load(pt_file)
        print('Loading %s dataset from %s, number of examples: %d' %
              (corpus_type, pt_file, len(dataset)))
        return dataset
    
    yield lazy_dataset_loader(TRAIN_DATASET, corpus_type)

In [6]:
# Peek the fisrt dataset to determine the data_type.
# (All datasets have the same data_type).

first_dataset = next(lazily_load_dataset("train"))
data_type = first_dataset.data_type

Loading train dataset from /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/boxscore-data/preprocess/roto.train.1.pt, number of examples: 3371


In [7]:
first_dataset

In [8]:
first_dataset.examples[0].__dict__.keys()

['src1_feat_0',
 'src1_feat_1',
 'src1_feat_2',
 'ptrs',
 'src2',
 'src1',
 'src_map',
 'tgt1_planning',
 'indices',
 'tgt2',
 'tgt1',
 'src2_feat_1',
 'src2_feat_0',
 'alignment',
 'src2_feat_2']

In [9]:
# Look into how 1st sample from roto.train.1.pt looks like - NBA
first_dataset.examples[0].__dict__['src1_feat_0']

(u'<blank>',
 u'<blank>',
 u'<blank>',
 u'<blank>',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kri

In [9]:
first_dataset.examples[0].__dict__['src2_feat_0']

(u'Celtics',
 u'Celtics',
 u'Celtics',
 u'Isaiah_Thomas',
 u'Isaiah_Thomas',
 u'Isaiah_Thomas',
 u'Al_Horford',
 u'Al_Horford',
 u'Al_Horford',
 u'Al_Horford',
 u'Al_Horford',
 u'Al_Horford',
 u'Marcus_Smart',
 u'Marcus_Smart',
 u'Marcus_Smart',
 u'Marcus_Smart',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Kristaps_Porzingis',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Carmelo_Anthony',
 u'Derrick_Rose',
 u'Derrick_Rose',
 u'Derrick_Rose')

In [10]:
def load_fields(dataset, data_type, checkpoint=None):
    
    fields = onmt.io.load_fields_from_vocab(torch.load(VOCAB_FILE), data_type)
    
    fields = dict([(k, f) for (k, f) in fields.items()
                   if k in dataset.examples[0].__dict__])

    if data_type == 'text' or data_type == 'box':
        print(' * vocabulary size. source1 = %d; target1 = %d, source2 = %d; target2 = %d' %
              (len(fields['src1'].vocab), len(fields['tgt1'].vocab), len(fields['src2'].vocab), len(fields['tgt2'].vocab)))

    return fields

In [11]:
# Load fields generated from preprocess phase.
fields = load_fields(first_dataset, data_type)

 * vocabulary size. source1 = 1164; target1 = 391, source2 = 956; target2 = 9902


In [12]:
fields

{'alignment': <torchtext.data.field.Field at 0x7f75411a2d50>,
 'indices': <torchtext.data.field.Field at 0x7f75411a2dd0>,
 'ptrs': <torchtext.data.field.Field at 0x7f75411a2d90>,
 'src1': <onmt.io.BoxField.BoxField at 0x7f752431fed0>,
 'src1_feat_0': <onmt.io.BoxField.BoxField at 0x7f75411a2ad0>,
 'src1_feat_1': <onmt.io.BoxField.BoxField at 0x7f75411a2b10>,
 'src1_feat_2': <onmt.io.BoxField.BoxField at 0x7f75411a2b50>,
 'src2': <torchtext.data.field.Field at 0x7f75411a2bd0>,
 'src2_feat_0': <torchtext.data.field.Field at 0x7f75411a2c50>,
 'src2_feat_1': <torchtext.data.field.Field at 0x7f75411a2c90>,
 'src2_feat_2': <torchtext.data.field.Field at 0x7f75411a2cd0>,
 'src_map': <torchtext.data.field.Field at 0x7f75411a2d10>,
 'tgt1': <torchtext.data.field.Field at 0x7f75411a2b90>,
 'tgt1_planning': <onmt.io.BoxField.BoxField at 0x7f75411a2a90>,
 'tgt2': <torchtext.data.field.Field at 0x7f75411a2c10>}

In [16]:
len(fields["tgt1"].vocab)

391

In [17]:
fields["tgt1"]

In [18]:
first_dataset.examples[0].__dict__['tgt1']

(u'604',
 u'605',
 u'600',
 u'398',
 u'399',
 u'380',
 u'354',
 u'355',
 u'336',
 u'348',
 u'349',
 u'351',
 u'442',
 u'443',
 u'437',
 u'424',
 u'46',
 u'47',
 u'28',
 u'40',
 u'24',
 u'25',
 u'6',
 u'112',
 u'113',
 u'94')

In [20]:
with open('bayer_dataset/rotowire/train_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    nba_pointerplan_train = reader.readlines()                               

In [21]:
nba_pointerplan_train[0]

'604 605 600 398 399 380 354 355 336 348 349 351 442 443 437 424 46 47 28 40 24 25 6 112 113 94\n'

In [22]:
first_dataset.examples[0].__dict__['tgt1']

(u'604',
 u'605',
 u'600',
 u'398',
 u'399',
 u'380',
 u'354',
 u'355',
 u'336',
 u'348',
 u'349',
 u'351',
 u'442',
 u'443',
 u'437',
 u'424',
 u'46',
 u'47',
 u'28',
 u'40',
 u'24',
 u'25',
 u'6',
 u'112',
 u'113',
 u'94')

In [23]:
def collect_report_features(fields):
    src_features = onmt.io.collect_features(fields, side='src1')
    tgt_features = onmt.io.collect_features(fields, side='tgt1')

    for j, feat in enumerate(src_features):
        print(' * src feature %d size = %d' % (j, len(fields[feat].vocab)))
    for j, feat in enumerate(tgt_features):
        print(' * tgt feature %d size = %d' % (j, len(fields[feat].vocab)))

# Report src/tgt features.
collect_report_features(fields)

 * src feature 0 size = 702
 * src feature 1 size = 39
 * src feature 2 size = 4


In [24]:
first_dataset.examples[0].__dict__['src1_feat_2']

(u'<blank>',
 u'<blank>',
 u'<blank>',
 u'<blank>',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME',
 u'HOME'

In [ ]:
# Build model.
model1, model2 = build_model(model_opt, opt, fields, checkpoint)
tally_parameters(model1)
tally_parameters(model2)
check_save_model_path()

Loading train dataset from /home/ruslan_yermakov/nlg-ra/reproducibility/data2text-plan-py/bayer_dataset/preprocess/roto.train.1.pt, number of examples: 1000
 * vocabulary size. source1 = 4660; target1 = 105, source2 = 4658; target2 = 7382
 * src feature 0 size = 3
 * src feature 1 size = 22
 * src feature 2 size = 4